In [2]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

from utils import list_oai_models

openai.api_type = "azure"
openai.api_version = "2022-12-01"

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

rg = os.getenv("resource_group")
svc = os.getenv("azure_openai_instance")

df = list_oai_models(os.getenv("mysubscription"), rg,svc)

try:
  response = openai.Completion.create(
    engine=df.loc[0, "display_name"],
    prompt="### Postgres SQL tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Salary_Payments(id, employee_id, amount, date)\n#\n### A query to list the names of the departments which employed more than 10 employees in the last 3 months\n\ query: SELECT",
    temperature=0,
    max_tokens=150,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["#",";"])

  print(response.choices[0].text)
except Exception as e:
    print(e)

Incomplete environment configuration. These variables are set: AZURE_USERNAME, AZURE_PASSWORD


 name FROM Salary_Payments WHERE employee_id > 10


In [3]:
try:
  response = openai.Completion.create(
    engine=df.loc[0,"display_name"],
    prompt="Can you explain what does this code do?\n#\n# ###\n\
    Code:\n\
    SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n#\n#\
    Answer:\n# ",
    max_tokens=250,
    stop=["#",";"])

  print(response.choices[0].text)
except Exception as e:
    print(e)

  This code selects all the names of the departments in the database, and then the names of the employees who work in those departments, and groups them by the number of times their name appears in the Salary_Payments table. If the number of times their name appears is more than 10, it then displays those names in a sorted list.
